# Self Organising Map Challenge

## The Kohonen Network

The Kohonen Self Organising Map (SOM) provides a data visualization technique which helps to understand high dimensional data by reducing the dimensions of data to a map. SOM also represents clustering concept by grouping similar data together.

Unlike other learning technique in neural networks, training a SOM requires no target vector. A SOM learns to classify the training data without any external supervision.

![Network](http://www.pitt.edu/~is2470pb/Spring05/FinalProjects/Group1a/tutorial/kohonen1.gif)

### Structure
A network has a width and a height that descibes the grid of nodes.  For example, the grid may be 4x4, and so there would be 16 nodes.

Each node has a weight for each value in the input vector.  A weight is simply a float value that the node multiplies the input value by to determine how influential it is (see below)

Each node has a set of weights that match the size of the input vector.  For example, if the input vector has 10 elements, each node would have 10 weights.

### Training 
To train the network

1. Each node's weights are initialized.
2. We enumerate through the training data for some number of iterations (repeating if necessary).  The current value we are training against will be referred to as the `current input vector`
3. Every node is examined to calculate which one's weights are most like the input vector. The winning node is commonly known as the Best Matching Unit (BMU).
4. The radius of the neighbourhood of the BMU is now calculated. This is a value that starts large, typically set to the 'radius' of the lattice,  but diminishes each time-step. Any nodes found within this radius are deemed to be inside the BMU's neighbourhood.
5. Each neighbouring node's (the nodes found in step 4) weights are adjusted to make them more like the input vector. The closer a node is to the BMU, the more its weights get altered.
6. Go to step 2 until we've completed N iterations.
    

### Calculating the Best Matching Unit (BMU)

To determine the best matching unit, one method is to iterate through all the nodes and calculate the Euclidean distance between each node's weight vector and the current input vector. The node with a weight vector closest to the input vector is tagged as the BMU.

The Euclidean distance $\mathsf{distance}_{i}$ (from the input vector $V$ to the $i$th node's weights $W_i$)is given as (using Pythagoras):

$$ \mathsf{distance}_{i}=\sqrt{\sum_{k=0}^{k=n}(V_k - W_{i_k})^2}$$

where V is the current input vector and $W_i$ is the node's weight vector.  $n$ is the size of the input & weight vector.

*Note*: $V$ and $W$ are vectors.  $V$ is the input vector, and $W_i$ is the weight vector of the $i$th node.  $V_k$ and $W_{i_k}$ represent the $k$'th value within those vectors.  

The BMU is the node with the minimal distance for the current input vector

### Calculating the Neighbourhood Radius

The next step is to calculate which of the other nodes are within the BMU's neighbourhood. All these nodes will have their weight vectors altered.

First we calculate what the radius of the neighbourhood should be and then use Pythagoras to determine if each node is within the radial distance or not.

A unique feature of the Kohonen learning algorithm is that the area of the neighbourhood shrinks over time. To do this we use the exponential decay function:

Given a desired number of training iterations $n$:
$$n_{\mathsf{max iterations}} = 100$$

Calculate the radius $\sigma_t$ at iteration number $t$:

$$\sigma_t = \sigma_0 \exp\left(- \frac{t}{\lambda} \right) \qquad t = 1,2,3,4... $$

Where $\sigma_0$ denotes the neighbourhood radius at iteration $t=0$, $t$ is the current iteration. We define $\sigma_0$ (the initial radius) and $\lambda$ (the time constant) as below:

$$\sigma_0 = \frac{\max(width,height)}{2} \qquad \lambda = \frac{n_{\mathsf{max iterations}}}{\log(\sigma_0)} $$

Where $width$ & $height$ are the width and height of the grid.

### Calculating the Learning Rate

We define the initial leanring rate $\alpha_0$ at iteration $t = 0$ as:
$$\alpha_0 = 0.1$$

So, we can calculate the learning rate at a given iteration t as:

$$\alpha_t = \alpha_0 \exp \left(- \frac{t}{\lambda} \right) $$

where $t$ is the iteration number, $\lambda$ is the time constant (calculated above)
        
### Calculating the Influence

As well as the learning rate, we need to calculate the influence $\theta_t$ of the learning/training at a given iteration $t$.  

So for each node, we need to caclulate the euclidean distance $d_i$ from the BMU to that node.  Similar to when we calculate the distance to find the BMU, we use Pythagoras.  The current ($i$th) node's x position is given by $x(W_i)$, and the BMU's x position is, likewise, given by $x(Z)$.  Similarly, $y()$ returns the y position of a node.

$$ d_{i}=\sqrt{(x(W_i) - x(Z))^2 + (y(W_i) - y(Z))^2} $$

Then, the influence decays over time according to:

$$\theta_t = \exp \left( - \frac{d_{i}^2}{2\sigma_t^2} \right) $$

Where $\sigma_t$ is the neighbourhood radius at iteration $t$ as calculated above. 

Note: You will need to come up with an approach to x() and y().


### Updating the Weights

To update the weights of a given node, we use:

$$W_{i_{t+1}} = W_{i_t} + \alpha_t \theta_t (V_t - W_{i_t})$$
        
So $W_{i_{t+1}}$ is the new value of the weight for the $i$th node, $V_t$ is the current value of the training data, $W_{i_t}$ is the current weight and $\alpha_t$ and $\theta_t$ are the learning rate and influence calculated above.

*Note*: the $W$ and $V$ are vectors 

## Challenge

Sam has written an implementation of a Self Organising Map. Consider the following criteria when assessing Sam's code:

- Could the code be made more efficient? A literal interpretation of the instructions above is not necessary.
  - Using too many python for loops. Must us numpy and vectors. This will also allow to use GPU in case that is available **Saad**
- Is the code best structured for later use by other developers and in anticipation of productionisation?
  - Nope, with this code it is very hard to understand what is happening at a glance
    - Make a class
    - Make internal functions for BMU, neighbourhood radius, learning rate and influence
- How would you approach productionising this application?
- Anything else you think is relevant.

In [37]:
# kohonen.py
import matplotlib.pyplot as plt
import numpy as np


def train(input_data, n_max_iterations, width, height, init_weights=None):
    σ0 = max(width, height) / 2
    α0 = 0.1
    weights = (
        init_weights
        if init_weights is not None
        else np.random.random((width, height, 3))
    )
    λ = n_max_iterations / np.log(σ0)
    for t in range(n_max_iterations):
        σt = σ0 * np.exp(-t / λ)
        αt = α0 * np.exp(-t / λ)
        for vt in input_data:
            bmu = np.argmin(np.sum((weights - vt) ** 2, axis=2))
            bmu_x, bmu_y = np.unravel_index(bmu, (width, height))
            for x in range(width):
                for y in range(height):
                    di = np.sqrt(((x - bmu_x) ** 2) + ((y - bmu_y) ** 2))
                    θt = np.exp(-(di**2) / (2 * (σt**2)))
                    weights[x, y] += αt * θt * (vt - weights[x, y])
    return weights

## Increasing efficiency 

- Use vector broad casting
- numpy vector operations instead of loops
- Update can be done without looping for x and y with meshgrid and broadcast

In [42]:
def e_train(input_data, n_max_iterations, width, height, init_weights=None):
    init_learning_rate = 0.1
    init_radius = max(width, height) / 2
    time_constant = n_max_iterations / np.log(init_radius)
    
    weights = (
        init_weights
        if init_weights is not None
        else np.random.random((width, height, input_data.shape[-1]))
    )
    coord_x, coord_y = np.meshgrid(np.arange(width), np.arange(height), indexing="ij")
    for t in range(n_max_iterations):
        current_radius = init_radius * np.exp(-t / time_constant)
        current_learning_rate = init_learning_rate * np.exp(-t / time_constant)
        for vt in input_data:
            bmu = np.argmin(np.sum((weights - vt) ** 2, axis=2))
            bmu_x, bmu_y = np.unravel_index(bmu, (width, height))

            influence = np.sqrt((coord_x - bmu_x) ** 2 + (coord_y - bmu_y) ** 2)
            influence_decay = np.exp(-(influence**2) / (2 * current_radius**2))
            # broadcasting
            influence_decay = influence_decay.reshape(
                influence_decay.shape + (1,) * (weights.ndim - influence_decay.ndim)
            )

            weights += current_learning_rate * influence_decay * (vt - weights)
    return weights

#### Some helper code

An `Instrument` class to create timer and logger functions

In [82]:
import time
from typing import Callable, Optional
from dataclasses import dataclass, field
from enum import Enum

class LogLevel:
    INFO="INFO"
    DEBUG="DEBUG"
    ERROR="ERROR"
    TIME="TIME"
    METRIC="METRIC"
    PARAM="PARAM"

@dataclass
class Log:
    level: LogLevel
    message: str
    data: Optional[dict] = None
    timestamp: Optional[str] = field(default_factory= lambda: time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))   
    
    def __str__(self):
        return f"{self.timestamp} [{self.level}] {self.message}"
        

class Telemetry:
    def __init__(self, log_exporter: Callable[[Log], None]=lambda log: print(str(log))):
        self.__log_exporter = log_exporter

    def log(self, message: str, data: Optional[dict] = None, level: LogLevel = LogLevel.INFO):
        self.__log_exporter(Log(
            level=level,
            data=data,
            message=message,
        ))
    
    def logger(self, func):
        """Decorator that wraps a function to log its execution details."""
        def wrapper(*args, **kwargs):
            try:
                start_time = time.time()
                self.log(
                    message=f"Entering {func.__name__}",
                    data=dict(
                        function_name=func.__name__,
                        args=args,
                        kwargs=kwargs
                    ),
                    level=LogLevel.DEBUG
                )
                result = func(*args, **kwargs)
                self.log(
                    message=f"Exiting {func.__name__}",
                    data=dict(
                        function_name=func.__name__,
                        result=result
                    ),
                    level=LogLevel.DEBUG
                )
                end_time = time.time()
                elapsed_time = end_time - start_time
                self.log(
                    f"Execution time of {func.__name__}: {elapsed_time:.4f} seconds", 
                    data=dict(
                        function_name=func.__name__,
                        start_time=start_time,
                        elapsed_time=elapsed_time,
                    ),
                    level=LogLevel.TIME
                )
                return result
            except Exception as e:
                self.log(f"Error in {func.__name__}: {e}", dict(error=e), level="ERROR")
                raise e
        return wrapper

### Making more production worthy code

In [144]:
from typing import Tuple, Optional
import numpy as np


class KohonenMap:
    def __init__(
        self,
        width: int,
        height: int,
        input_dim: int,
        max_iterations: int = 1000,
        learning_rate: float = 0.1,
        weights: Optional[np.ndarray] = None,
        telemetry: Telemetry = Telemetry(),
    ):
        self.width = width
        self.height = height
        self.input_dim = input_dim
        self.learning_rate = learning_rate
        self.weights = (
            weights
            if weights is not None
            else np.random.random((width, height, input_dim))
        )
        assert self.weights.shape == (width, height, input_dim), f"The weights must be of shape {(width, height, input_dim)}. The given is {self.weights.shape}"
        self.max_iterations = max_iterations
        self.meshgrid = np.meshgrid(np.arange(width), np.arange(height), indexing="ij")
        self.init_radius = max(width, height) / 2
        self.time_constant = max_iterations / np.log(self.init_radius)
        self.telemetry = telemetry

    def __telemtry(self, func):
        return self.telemetry.logger(func)
    
    def __get_bmu(self, vector: np.ndarray) -> Tuple[int, int]:
        bmu = np.argmin(np.sum((self.weights - vector) ** 2, axis=2))
        return np.unravel_index(bmu, (self.width, self.height))

    def __calculate_influence(
        self, bmu: Tuple[int, int], current_radius: float
    ) -> np.ndarray:
        bmu_x, bmu_y = bmu
        coord_x, coord_y = self.meshgrid
        influence = np.sqrt((coord_x - bmu_x) ** 2 + (coord_y - bmu_y) ** 2)
        influence_decay = np.exp(-(influence**2) / (2 * current_radius**2))
        return influence_decay.reshape(
            influence_decay.shape + (1,) * (self.weights.ndim - influence_decay.ndim)
        )

    def __instrumented_data_iterator(self, input_data: np.ndarray, weight_update_func: Callable[[int, np.ndarray], None]):
        instrumentation = {}
        instrumentation["total_iterations"] = self.max_iterations * input_data.shape[0]
        instrumentation["total_time_elapsed"] = 0
        for iteration in range(self.max_iterations):
            for index, vector in enumerate(input_data):
                instrumentation["start_time"] = time.time()
                weight_update_func(iteration, vector) 
                instrumentation["iteration"] = (iteration * input_data.shape[0]) + (index+1)
                instrumentation["progress"] = instrumentation["iteration"]/instrumentation["total_iterations"]
                instrumentation["elapsed_time"] = time.time() - instrumentation["start_time"]
                instrumentation["total_time_elapsed"] += instrumentation["elapsed_time"]
                instrumentation["time_remaining"] = (instrumentation["total_time_elapsed"]/instrumentation["iteration"]) * (instrumentation["total_iterations"] - instrumentation["iteration"])
                self.telemetry.log(
                    message=f'''Progress = {(instrumentation["progress"] * 100):.2f}% ({instrumentation["iteration"]}/{instrumentation["total_iterations"]}) | Iteration time = {instrumentation["elapsed_time"]:.5f}s | Approx. time remaining = {instrumentation["time_remaining"]}s''',
                    data=instrumentation
                )
        

    def __train(self, input_data: np.ndarray):
        assert input_data.shape[-1] == self.input_dim, f"The input_data must be of shape (N, {self.input_dim}). The given is {input_data.shape}"
        def weight_update_function(iteration: int, vector: np.ndarray) -> None:
            current_radius = self.init_radius * np.exp(-iteration / self.time_constant)
            current_learning_rate = self.learning_rate * np.exp(-iteration / self.time_constant)
            bmu = self.__telemtry(self.__get_bmu)(vector)            
            influence_decay = self.__telemtry(self.__calculate_influence)(bmu, current_radius)
            self.weights += current_learning_rate * influence_decay * (vector - self.weights)
                    
        self.__instrumented_data_iterator(input_data, weight_update_function)
        return self.weights
        

    def train(self, input_data: np.ndarray):
        return self.__telemtry(self.__train)(input_data)        


def p_train(input_data, n_max_iterations, width, height, init_weights=None):
    kmap = KohonenMap(
        width=width,
        height=height,
        input_dim=3,
        max_iterations=n_max_iterations,
        weights=init_weights,
        learning_rate=0.2
    )

    kmap.train(input_data)
    return kmap.weights

In [151]:
if __name__ == "__main__":
    # Generate data
    w, h = (400, 400)
    iterations = 100
    input_data = np.random.random((40, 3))
    init_weights = np.random.random((w, h, 3))
    plt.imsave(f"init{w}.png", init_weights)
    
    image_data = p_train(input_data, iterations, w, h, init_weights)
    plt.imsave(f"p{w}.png", image_data)

    # image_data = e_train(input_data, iterations, w, h, init_weights)
    # plt.imsave("e100.png", image_data)

    # image_data = train(input_data, iterations, w, h, init_weights)
    # plt.imsave("100.png", image_data)

    # Generate data
    # input_data = np.random.random((10,3))
    # image_data = train(input_data, 1000, 100, 100)

    # plt.imsave('1000.png', image_data)

2024-06-11 00:28:35 [DEBUG] Entering _train
2024-06-11 00:28:35 [DEBUG] Entering _get_bmu
2024-06-11 00:28:35 [DEBUG] Exiting _get_bmu
2024-06-11 00:28:35 [TIME] Execution time of _get_bmu: 0.0059 seconds
2024-06-11 00:28:35 [DEBUG] Entering _calculate_influence
2024-06-11 00:28:35 [DEBUG] Exiting _calculate_influence
2024-06-11 00:28:35 [TIME] Execution time of _calculate_influence: 0.0146 seconds
2024-06-11 00:28:35 [INFO] Progress = 0.03% (1/4000) | Iteration time = 0.03581s | Approx. time remaining = 143.20035123825073s
2024-06-11 00:28:35 [DEBUG] Entering _get_bmu
2024-06-11 00:28:35 [DEBUG] Exiting _get_bmu
2024-06-11 00:28:35 [TIME] Execution time of _get_bmu: 0.0045 seconds
2024-06-11 00:28:35 [DEBUG] Entering _calculate_influence
2024-06-11 00:28:35 [DEBUG] Exiting _calculate_influence
2024-06-11 00:28:35 [TIME] Execution time of _calculate_influence: 0.0011 seconds
2024-06-11 00:28:35 [INFO] Progress = 0.05% (2/4000) | Iteration time = 0.00883s | Approx. time remaining = 89.2